In [1]:
import os
import time
import requests
import pandas as pd

import config as cfg

In [6]:
import base64

header_string = cfg.ravkey['username'] + ':' + cfg.ravkey['password']
auth_header_bytes = base64.urlsafe_b64encode(header_string.encode('utf-8'))
auth_header = str(auth_header_bytes, 'utf-8')

headers = { "Authorization": "Basic " + auth_header }
requests.get('https://api.ravelry.com/current_user.json', headers = headers)

<Response [403]>